# Speed Experiments (Gear: Drive)

## Imports

In [ ]:
from pathlib import Path
from datetime import date
import os
import subprocess
from datetime import datetime
from copy import deepcopy
from revcan.reverse_engineering.models.experiment import Experiment

## Create Necessary Experiment Files

### TODO: 
Set & check the following parameters:
- **Car Model File Name**: Specify the name of the car model file (`{model}_{vin}_{date-created}.json`).
- **Car Model Directory**: Check the directory path where the car model file is stored (Default: `../../data/car_metadata`).
- **Experiment Folder**: Set the path for the experiment folder where the results will be saved (Default: `../../data/experiments/{car}/{experiment}/{date-today}`).

In [ ]:
car_model_file_name = "model_vin_date-created.json" # TODO: Set correct car model file name
car_model_dir = "../../data/car_metadata" # TODO: Check directory of car model file
experiment_folder = "../../data/experiments/car/speed-drive/" + date.today().strftime("%Y-%m-%d") # TODO: Set directory where experiment files should be saved to

car_model_file = f"{car_model_dir}/{car_model_file_name}"
car_model_file_path = os.path.abspath(car_model_file)
Path(experiment_folder).mkdir(parents=True, exist_ok=True)

speeds=["10","20","30","40","50","60","70","80","90","100","110","120"]

for speed in speeds:

    experiment_file_path = os.path.join(experiment_folder, speed+"kmh.json")
    experiment_name = f"Speed experiment {speed} km/h"
    experiment_description = "speed signal discovery using 13 different speeds."
    signal_selection ="all"
    print(f"Experiment file path: {experiment_file_path}")
    !python ../scripts_for_doip_new/04_create_experiment.py --car_model_file_path {car_model_file_path} --experiment_file_path '{experiment_file_path}' --experiment_name '{experiment_name}' --experiment_description '{experiment_description}' --signal_selection {signal_selection}


## [Optional]: Set Experiment Names manually

In [ ]:
car_model_file_name = "model_vin_date-created.json" # TODO: Set correct car model file name
car_model_dir = "../../data/car_metadata" # TODO: Check directory of car model file
experiment_folder = "../../data/experiments/car/speed-drive/2025-02-25_10_34_26"
speeds=["10","20","30","40","50","60","70","80","90","100","110","120"]

car_model_file = f"{car_model_dir}/{car_model_file_name}"
car_model_file_path = os.path.abspath(car_model_file)


## Measurements

## Measurement - Speed x km/h
**Before** executing the next Cell:

- Put the car in **Drive**

- Hold the speed of **x** km/h by pressing the **gas pedal**

Start measurement


In order to apply **pre-filtering** strategy:

After **2** measurements: 

- execute the Filter Cells below and **return**
- complete the remaining measurements.
- Continue normal flow of cells.



In [ ]:
speed = 10 #TODO set value which is tested
sample_number = 4 #TODO set number of samples to record
experiment_file_path = os.path.join(experiment_folder, speed+"kmh.json")
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path "{experiment_path}" --activate_logging true --num_samples "{sample_number}"

## Evaluation

## [Optional] Display Experiment Metadata

In [ ]:
# Display Experiment
for speed in speeds:
    experiment_file = os.path.join(experiment_folder, f"{speed}kmh.json")
    !python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"


## Load Experiments

Load all relevant experiments needed for this analysis

### Imports for Evaluation

In [ ]:
experiment_files = {}
experiments = {}

for speed in speeds:
    experiment_files[speed] = os.path.join(experiment_folder, f"{speed}kmh.json")
    experiments[speed] = Experiment.load(experiment_files[speed])
pass

Add ground truth data for steering wheel angle. By experiment design these are fixed values, and are added manually in this case

In [ ]:
from datetime import timedelta
import datetime
from typing import List
from copy import deepcopy
from revcan.reverse_engineering.models.experiment import Extern_Signal, Value

for speed in speeds:
    experiments[speed].external_measurements.append(Extern_Signal( name = "Speed", id = 1, values = []))
    values:List[Value] = []
    for value in experiments[speed].measurements[0].values:
        values.append(Value(value= [speed], time= value.time))
    experiments[speed].external_measurements[0].values = deepcopy(values)
    print(f"Experiment speed {speed} km/h: {len(experiments[speed].external_measurements[0].values)} ground truth values")

## Filter

Concat all experiments

In [ ]:
combined_experiment = deepcopy(experiments[speeds[0]])
combined_experiment.name = f"Non constant singals for speeds experiment"
combined_experiment.description = f"Non constant signals for speeds experiment with {len(speeds)} different speeds."

for speed in speeds[1:]:
    combined_experiment.experiment_runtime_seconds += experiments[speed].experiment_runtime_seconds
    for measurement in experiments[speed].measurements:
        for x in combined_experiment.measurements:
            if x.serverid == measurement.serverid:
                if x.did == measurement.did:
                    x.values.extend(measurement.values)
                    break

Keep only non constant signals

In [ ]:
number_measurements = len(combined_experiment.measurements)
print(f"Experiment contains {number_measurements} measurements before filtering", )
print("Applaying remove constant values filter")

combined_experiment.keep_non_constant_signals(combined_experiment)

number_measurements = len(combined_experiment.measurements)
print(f"Experiment contains {number_measurements} measurements after filtering", )


Remove all previously found constant signals from individual experiments

In [ ]:

for speed in speeds:
    experiments[speed].keep_signals_by_list(signals_list=combined_experiment.measurements)
    experiment_file = os.path.join(experiment_folder, speed+"kmh.json")
    experiments[speed].save(f"{experiment_file}")
    !python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"


Save combined experiment file

In [ ]:
combined_experiment_file = os.path.join(experiment_folder, f"combined.json")
combined_experiment.save(f"{combined_experiment_file}")
!python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{combined_experiment_file}"